This notebook explores the PPP data from https://data.sba.gov/dataset/ppp-foia/resource/6dd520ff-c211-404f-b7eb-29784cd4dbda?inner_span=True.

The goal is to see how PPP disbursements compared to the number of US religious congs from the US religious census: https://www.usreligioncensus.org/, with a special focus on whether black church congregrations didn't benefit from the PPP program as much as other groups.

One major difficulty in this analysis is that the Census and PPP data use different names for the groups. The PPE uses individual congregrations by their legal (TBA) names, but the Census uses scholarly / official names, which can be different. For example, the Census identifies five groups as "Anglican":
    Anglican Catholic Church
    Anglican Church in America
    Anglican Church in North America
    Anglican Province of America
    Anglican Province of Christ the King
 
The "Anglican Church in NA" has the most congregrations (873), while the other four have a total of less than 200. None of these report the number of adherents. But most "Anglican" churches names don't use these names. To which group does "St. Paul's Anglican Church" belong? The PPP data doesn't allow us to determine this.

Similarly, the Census has data about different Jewish groups ("Reform Judaism", "Conservative Judaism"), but many synagogues don't use these terms in their names: is "Temple Israel" a reform, conservative, or some other branch of Judaism?

In this analysis, I have endeavored to address these issues and while there are doubtless errors, I think the overall conclusions are correct.


In [1]:
import pandas as pd
pd.options.display.max_colwidth = 400
pd.options.display.max_columns = 90

In [ ]:
import glob
workDir = '/home/alp/Google Drive/Python/analysis/PPP_analysis/'

file_names = sorted(glob.glob(workDir+'public_up_to*.csv'))
firstFile = 'public_150k_plus_220930.csv'

colList = ['BorrowerName','BorrowerState','BorrowerZip','CurrentApprovalAmount','JobsReported','NAICSCode','Race','Ethnicity','BusinessType','ForgivenessAmount']

df = pd.read_csv(workDir+firstFile, usecols = colList)

for file in file_names:
    oldDF = df
    #print(file)
    if firstFile not in file:
        newDF = pd.read_csv(file, usecols = colList)
        df = pd.concat([oldDF,newDF])

df.info()        

In [4]:
print(f'Number of records in the dataframe is {len(df):,}')

Number of records in the dataframe is 11,468,438


How to separate out the religious organizations? 

The PPP incldues a field for NAICS codes. According to the __[US Census](https://www.census.gov/naics/?input=813110&year=2022&details=813110)__ 813110 is the NAICS code for religious organizations:
> "This industry comprises (1) establishments primarily engaged in operating religious organizations, such as churches, religious temples, mosques, and monasteries, and/or (2) establishments primarily engaged in administering an organized religion or promoting religious activities."


In [5]:
print('pct of records disbursed to religious organizations:')
len(df[df['NAICSCode'] == 813110])/len(df)*100

pct of records disbursed to religious organizations:


1.0695964001374905

We will create a new DF just for religious organizations and clean it a bit.

In [6]:
rel_df  = df[df['NAICSCode'] == 813110]
rel_df = rel_df.dropna(subset = ['BorrowerName'])
len(rel_df)

122664

The PPP records a wide variety of different types of business, even among the religious organizations. It's hard to know if some of these should be excluded from the analysis (especially since there have been news reports of folks scamming the PPP system __[NBC](https://www.nbcnews.com/politics/justice-department/biggest-fraud-generation-looting-covid-relief-program-known-ppp-n1279664)__ and __[NPR](https://www.npr.org/2022/10/12/1128207464/ppp-loans-loan-forgiveness-small-business)__. But since I have no way of determining if any of the loans reported in the PPP database are fraudulent or if some categories (such as "Sole Proprietorship") might have higher rates of fraud than others (such as "Corporation"), I will retain all of them for this analysis.

In [ ]:
rel_df['BusinessType'].value_counts()

Now, let's create a subset df that extracts just the religious organizations that are among the family of churchs that are traditional black churches. 

According to Frontline(https://www.pbs.org/wgbh/americanexperience/features/godinamerica-black-church/):
```
Today "the black church" is widely understood to include the following seven major black Protestant denominations: 
    the National Baptist Convention, 
    the National Baptist Convention of America, 
    the Progressive National Convention, 
    the African Methodist Episcopal Church, 
    the African Methodist Episcopal Zion Church, 
    the Christian Methodist Episcopal Church and
    the Church of God in Christ.
```
So let's look for those seven denoms.

In [8]:
bc_fam = rel_df[rel_df['BorrowerName'].str.contains('NATIONAL BAPTIST | PROGRESSIVE NATIONAL | AFRICAN METHODIST | METHODIST EPISCOPAL | CHURCH OF GOD IN CHRIST')
                        ]
len(bc_fam)

552

In [9]:
# percent of religion loans represted by BC fam churches:
len(bc_fam)/len(rel_df)*100

0.4500097828213657

In the PPE data, 122,664 loans were provided to religious organizations.

Of this, 552 were provided to congs that may be one of the seven historic BCs. 
This is 0.45% of the total. 

But in the Census, the 7 BC families comprise about 3.77% of all congregrations in the USA.

This seems to be a big disparity. Part of the explanation is probably the difficulty of applying for PPO funds. According to media reports, applying for PPP requried familarity with the banking industry, legal knowledge helped, etc. Folks who are wealthier and better educated, then, should have found it easier to get PPP loans.

As a check on this, let's examine the data for the Episcopal Church, Hindu, and Jewish groups. These groups have higher-than-average education and income levels; see the __[PEW religious landscape survey](https://www.pewresearch.org/religion/religious-landscape-study/)__ data for information about this. What percent of the national group do they represent? Did they get this amount from the PPO pool or a higher number?

We will start by creating dataframes for the three other groups and then examining how many loans they received.

In [36]:
j_fam = rel_df[(rel_df['BorrowerName'].str.contains("SYNAGOGUE | JEWISH | ISRAEL | BEIT | TORAH | B'NAI | ZEDEK | TZEDEK | ANSHE | MENORAH | KOL | HILLEL")) 
                        ]
# these search terms are, doubless, missing congregrations. Synagogues often don't have "synagogue" in their name.
# I looked at "JUDAH" but most of these also include Christian terms in their names ("Church", "Ministry", "Deliverance")
# There are a lot of "TEMPLE" names for Jewish congs, but TEMPLE is also used by many other groups.
# "ISRAEL" picks up some churchs, but it seems to be more common for Jewish congs.


In [38]:
episcop = rel_df[rel_df['BorrowerName'].str.contains('EPISCOPAL | ANGLICAN' )] 
# but this will pick up the AMEZ church, so we need to delete those rows

episcop = episcop.drop( episcop[episcop['BorrowerName'].str.contains('AFRICAN METHODIST')].index)

In [43]:
hindu = rel_df[rel_df['BorrowerName'].str.contains('HINDU | YOGA | SHREE | KRISHNA')]
# "Krishna" brings in ISKCON which some traditional Hindus may not regard as legitimate, but the Census appears to only have "Hindu Temples" without
# differentiating between them, so I'll include them here.

In [44]:
print(f'Number of PPE loans disbursed to Jewish congs: {len(j_fam)}')
print(f'Pct of religion loans made to J congs: {len(j_fam)/len(rel_df)*100:.2f}')
print(f'Number of PPE loans disbursed to Episcopal congs: {len(episcop)}')
print(f'Pct of religion loans made to E congs: {len(episcop)/len(rel_df)*100:.2f}')
print(f'Number of PPE loans disbursed to Hindu congs: {len(hindu)}')
print(f'Pct of religion loans made to H congs: {len(hindu)/len(rel_df)*100:.2f}')

Number of PPE loans disbursed to Jewish congs: 1179
Pct of religion loans made to J congs: 0.96
Number of PPE loans disbursed to Episcopal congs: 3034
Pct of religion loans made to E congs: 2.47
Number of PPE loans disbursed to Hindu congs: 201
Pct of religion loans made to H congs: 0.16


Summary:

| Relig group/fam | N PPP loans | Pct PPP loans | Pct All US cong | N loans if proportional |
| --- | --: | --: | --: | --: |
| Black Church | 552 | 0.45 | 3.77 | 4624 |
| Jewish | 1179 | 0.96 | 0.66 | 809 |
| Episcopal | 3034 | 2.47 | 2.03 | 3030 |
| Hindu | 201 | 0.16 | 0.51 | 625 |

This suggests that
- Episcopal groups received PPP loans almost exactly in proportion to their number in the Census.
- Jewish groups recevied *more* loans than would be proportional (+45%).
- Hindu groups received *many fewer* loans than would be proportional (-68%).
- Black churches reeived *many fewer* loans that would be proportional (-88%). 

But perhaps there is a difference in the
1. the size of the congregrations?
1. the number of jobs at risk?
1. the amount of the loans?

Data for the first can be found in the Census:

| Relig group/fam | N congs | N Adherents | Adh per Cong |
| --- | --: | --: | --: |
| Black Church | 13,457 | 4,602,235 | 342 |
| Jewish | 2,353 | 2,068,827 | 879 
| Episcopal | 7,226 | 1,576,611 | 218 |
| Hindu | 1,811 | 1,276,444 | 704 |

This shows that the average size of Episcopal congregrations is the smallest of the four groups, followed by Black churchs. Hindu and Jewish congregrations are considerably larger.

Let's look at the latter two questions:

In [54]:
totBC, totJ, totE, totH = bc_fam['JobsReported'].sum(), j_fam['JobsReported'].sum(), episcop['JobsReported'].sum(),hindu['JobsReported'].sum()
totBCfg, totJfg, totEfg, totHfg = bc_fam['ForgivenessAmount'].sum(), j_fam['ForgivenessAmount'].sum(), episcop['ForgivenessAmount'].sum(),hindu['ForgivenessAmount'].sum()

print(f'{"Rel Family":<12}{"Total N Loans":>15}{"Total Jobs":>12}{"Jobs/loan":>12}{"Total $ Forgiven":>20}{"Forgive/Cong":>15}{"Forgive/Job":>15}{"Forgive/loan":>15}')
print(f'{"Black church":<12}{len(bc_fam):>15n}{totBC:>12n}{(totBC/len(bc_fam)*100):>12,.0f}{totBCfg:>20,.2f}{(totBCfg/len(bc_fam)):>15,.2f}{(totBCfg/totBC):>15,.2f}{(totBCfg/len(bc_fam)):>15,.2f}')
print(f'{"Jewish":<12}{len(j_fam):>15n}{totJ:>12n}{(totJ/len(j_fam)*100):>12,.0f}{totJfg:>20,.2f}{(totJfg/len(j_fam)):>15,.2f}{(totJfg/totJ):>15,.2f}{(totJfg/len(j_fam)):>15,.2f}  ')
print(f'{"Episcopal":<12}{len(episcop):>15n}{totE:>12n}{(totE/len(episcop)*100):>12,.0f}{totEfg:>20,.2f}{(totJfg/len(episcop)):>15,.2f}{(totEfg/totE):>15,.2f}{(totJfg/len(episcop)):>15,.2f}  ')
print(f'{"Hindu":<12}{len(hindu):>15n}{totH:>12n}{(totH/len(hindu)*100):>12,.0f}{totHfg:>20,.2f}{(totHfg/len(hindu)):>15,.2f}{(totHfg/totH):>15,.2f}{(totJfg/len(hindu)):>15,.2f}  ')


Rel Family    Total N Loans  Total Jobs   Jobs/loan    Total $ Forgiven   Forgive/Cong    Forgive/Job   Forgive/loan
Black church            552        3664         664       16,732,818.05      30,313.08       4,566.82      30,313.08
Jewish                 1179       23284       1,975      149,484,804.95     126,789.49       6,420.07     126,789.49  
Episcopal              3034       28726         947      178,483,856.39      49,269.88       6,213.32      49,269.88  
Hindu                   201        1150         572        7,161,667.02      35,630.18       6,227.54     743,705.50  


The most interesting column in this table is the amount of `forgiveness provided per job`: there is remarkable similarity between Jewish, Episcopal, and Hindu religious organizations, with an average around $6,300. The amount forgiven to Black churchs is much lower.

The rest of the figures vary widely. Jewish and Episcopal groups seem to employ far more people than do the Black church and Hindu organizations and the numbers of people supported per loan reflect this. Since the PPP was mainly designed to support employemnt, it is not surprising that the amount forgiven is correspondingly higher.

**To return to the initial question: did black churchs recieve less support from the PPP program as some other groups?**

The answer to this is 'yes.' 
- If black churches had received loans in proportion to their representation in the Census, they would have received around 4,600 loans, rather than 552.
- If they received roughly the same amount of money per job, they would have received about $1,750 per job more than they did.

This data doesn't allow us to determine *why* Black churches received fewer loans and for smaller amounts.